In [1]:
import pandas as pd
import numpy as np


In [2]:
question_answer = pd.read_csv('csv/Posts.csv')

In [3]:
question_answer.set_index('Id', inplace=True)

In [4]:
answers = question_answer[question_answer['PostTypeId'] == 2]
questions = question_answer[question_answer['PostTypeId'] == 1]
# this observation has come from the fact that, PostTypeId == 2, have a parent Id, while those with 1 have answerCount field non empty, rest all ids are wikis etc.

In [ ]:
answerer = answers.groupby('OwnerUserId')['ParentId'].apply(list).to_dict()
answerer

In [ ]:
question_tags = questions['Tags'].apply(lambda x: list(filter(lambda x: x != '', x.split('|'))))
question_tags

In [ ]:
exploded_tags = question_tags.explode()
exploded_tags

In [ ]:
tags = exploded_tags.value_counts()
tags

In [ ]:
count_tag = tags.to_dict()
count_tag

In [ ]:
answers.isna().sum()
# 6822 answers are such that their owner is missing

In [11]:
count_answers_per_user = answers['OwnerUserId'].value_counts().to_dict()

In [ ]:
count_answers_per_user

In [13]:
#! wrong code, gives very wrong answers, design is the most used tag

# count_tag = dict()
# for tags in question_tags:
#   for tag in tags:
#     if tag not in count_tag:
#       count_tag[tag] = 0
#     count_tag[tag] += 1

In [ ]:
count_tag

In [15]:
tags_df = pd.read_csv('csv/Tags.csv')

In [ ]:
# tags_df.set_index('Id', inplace=True)
tag_cols = tags_df.columns
tag_cols

In [ ]:
req_tag_cols = ['Id', 'TagName']
drop_cols = list(filter(lambda x: x not in req_tag_cols, tag_cols))
drop_cols


In [ ]:
new_tags_df = tags_df.drop(drop_cols, axis=1)
new_tags_df

In [ ]:
new_tags_df.set_index('TagName', inplace=True)
new_tags_df

In [ ]:
tag_dict = new_tags_df.to_dict()['Id']
tag_dict

In [21]:
count_tag = {tag_dict[k]: v for k, v in count_tag.items() if k in tag_dict} # some tags not there in the tags.csv file, don't know why :(
# count_tag

In [ ]:
answerer_table = pd.DataFrame(count_answers_per_user.items(), columns=['UserId', 'AnsweredQuestionCount'])
answerer_table

In [ ]:
tags_table = pd.DataFrame(count_tag.items(), columns=['TagId', 'Count'])
tags_table

In [ ]:
sorted_tags_table = tags_table.sort_values(by='Count', ascending=False)
sorted_tags_table.head(3)

In [ ]:
sorted_answerer_table = answerer_table.sort_values(by='AnsweredQuestionCount', ascending=False)
sorted_answerer_table.head(3)

# Second Part

In [26]:
threshold = 20

In [ ]:
answerer_table = answerer_table[answerer_table['AnsweredQuestionCount'] >= threshold]
answerer_table

In [ ]:
tags_table = tags_table[tags_table['Count'] >= threshold]
tags_table

In [ ]:
print(tag_dict)

In [30]:
user_tag_count = {}
tags_table = tags_table.sort_values(['TagId'])
answerer_table = answerer_table.sort_values(['UserId'])
users = answerer_table['UserId'].values
tags = tags_table['TagId'].values
q_tag_map = question_tags.to_dict()


for user in users:
  tag_count = {tag : 0 for tag in tags}
  for question in answerer[user]:
    for tag in q_tag_map[question]:
      try:
        tag_count[tag_dict[tag]] += 1
      except KeyError:
        pass
  user_tag_count[user] = tag_count
del q_tag_map
del users
del tags

In [ ]:
user_tag_count

In [32]:
expert = pd.DataFrame.from_dict(user_tag_count, orient='index')
del user_tag_count

In [ ]:
expert

In [ ]:
expert.shape # 1163 users, 973 tags

# Third Part

In [ ]:
expert = expert.map(lambda x: x/3 if x < 15 else 5)
expert

In [ ]:
expert_matrix = expert.to_numpy()
expert_shape = expert_matrix.shape
expert

In [ ]:
expert_matrix[expert_matrix == 0] = np.nan
np.isnan(expert_matrix).sum()

In [ ]:
test_start = (int(0.85 *expert_shape[0] ), int(0.85*expert_shape[1]))
test_users, test_tags = test_start
test_start

In [ ]:
test_matrix = np.copy(expert_matrix[test_users:, test_tags:])
test_matrix.shape

In [40]:
train_matrix = np.copy(expert_matrix)
train_matrix[test_users:, test_tags:] = np.nan

In [ ]:
"""
Report the following about your utility matrix:
Summation value of the utility matrix
Highest row sum of the utility matrix
Highest column sum of the utility matrix

"""

sum_utility = np.nansum(expert_matrix)
highest_row_sum = np.nansum(expert_matrix, axis = 0).max()
highest_col_sum = np.nansum(expert_matrix, axis = 1).max()
sum_utility, highest_row_sum, highest_col_sum

In [ ]:
"""
Report the following for your train and test data
Summation value of the train matrix
Dimension of the test matrix
Summation value of test matrix
"""

sum_train = np.nansum(train_matrix)
dim_test = test_matrix.shape
sum_test = test_matrix.sum()
sum_train, dim_test, sum_test


# Fourth Part

In [43]:
test_users_start, test_tags_start = test_start

In [44]:
from abc import ABC, abstractmethod
import numpy as np
class CollaborativeFilter():
  def __init__(self, utility_matrix, function='weighted'):
    utility_zero_mask = utility_matrix == 0
    utility_matrix[utility_zero_mask] = np.nan
    self.utility_matrix = utility_matrix
    self.utility_matrix_filled = utility_matrix.copy()
    self.epsilon = 1e-9 # a small value to avoid division by zero
    
    self.final_rating_function = None
    if function == 'weighted':
      self.final_rating_function = self.weighted_average
    elif function=='regular_average':
      self.final_rating_function = self.average
    else:
      raise Exception("Function not allowed")
    
  
  def weighted_average(self, vector: np.ndarray, scores: np.ndarray):
    return np.dot(vector, scores)/(np.sum(scores) + self.epsilon)
  
  def average(self, vector: np.ndarray, scores: np.ndarray):
    return np.mean(vector)
    

  @abstractmethod
  def predict(self):
    pass
  
  
  @abstractmethod
  def compute_similarities(self):
    pass
  
  

In [45]:
class UserBasedCollaborativeFilter(CollaborativeFilter):
    def __init__(self,expert_matrix ,k , function):
      """
      utility_matrix : 2D numpy array
      k : int
      
      k is the number of similar users to consider for prediction
      utility_matrix is the matrix of user ratings, nan filled with 0
      """
      super().__init__(expert_matrix, function)
      self.k = k
    
    def compute_similarities(self, user_vector : np.ndarray):
      """
      user_vector : 1D numpy array
      """
      # self.utility_matrix has each column as a user and each row as an item
      utility_rating_means = np.nanmean(self.utility_matrix, axis=0)
      user_rating_mean = np.nanmean(user_vector)
      # subtract the mean rating of each user from their ratings
      utility_matrix_centered = self.utility_matrix - utility_rating_means
      
      # compute the cosine similarity between user_vector and each user
      user_vector_centered = user_vector - user_rating_mean
      
      # compute the norms of the vectors, which might have nans as well
      user_vector_norm = np.sqrt(np.nansum(user_vector_centered**2))
      utility_matrix_norm = np.sqrt(np.nansum(utility_matrix_centered**2, axis=1))
      # compute the dot product between user_vector and each user
      
      dot_product = np.nansum(utility_matrix_centered *user_vector_centered, axis = 1)
      # compute the cosine similarity
      similarities = dot_product/(user_vector_norm * utility_matrix_norm + self.epsilon)
      
      return similarities
    
    def find_k_nearest_users(self, user_vector : np.ndarray, i: int):
      """
      user_vector : 1D numpy array
      
      returns the indices of the k most similar users to user_vector
      """
      
      similarities = self.compute_similarities(user_vector)
      users_to_consider = np.where(~np.isnan(self.utility_matrix[:, i]))[0]
      k = min(self.k, len(users_to_consider))
      sorted_indices = np.argsort(similarities[users_to_consider])[-k:]
      users_to_consider = users_to_consider[sorted_indices]
      similarities = similarities[users_to_consider]
      return similarities, users_to_consider
    
    
    def predict(self, user_vector : np.ndarray, i: int):
      """
      user_vector : 1D numpy array
      i : int
      
      i is the index of the item to predict the rating for
      """
      
      # find the k most similar users who rated this item 
      similarities, users_to_consider = self.find_k_nearest_users(user_vector, i)
      # similarity_scores = similarities[users_to_consider]
      
      # predict the rating for the item
      prediction = self.final_rating_function(self.utility_matrix[users_to_consider, i], similarities)
      return prediction
      

In [46]:
def test_user_collaborator(train_matrix: np.ndarray, expert_matrix: np.ndarray, user_start: int, item_start: int, model: UserBasedCollaborativeFilter) -> float:
    rmse_loss = 0
    count = 0
    for i in range(user_start, expert_matrix.shape[0]):
        for j in range(item_start, expert_matrix.shape[1]):
            if not np.isnan(expert_matrix[i, j]):
                prediction = model.predict(train_matrix[i], j)
                # print(prediction, i, j)
                rmse_loss += (prediction - expert_matrix[i, j])**2
                count += 1
    rmse_loss = np.sqrt(rmse_loss/count)
    
    return rmse_loss


In [47]:
user_cf = UserBasedCollaborativeFilter(train_matrix, 5, 'regular_average')
# test_user_collaborator(train_matrix, expert_matrix, test_users_start, test_tags_start, user_cf)

In [48]:
user_cf = UserBasedCollaborativeFilter(train_matrix, 5, 'weighted')
# test_user_collaborator(train_matrix, expert_matrix, test_users_start, test_tags_start, user_cf)

In [49]:

class ItemBasedCollaborativeFilter(CollaborativeFilter):
    def __init__(self,expert_matrix ,k, function):
      """
      utility_matrix : 2D numpy array
      k : int
      
      k is the number of similar items to consider for prediction
      utility_matrix is the matrix of user ratings, nan filled with 0
      """
      super().__init__(expert_matrix, function)
      self.k = k
    
    def compute_similarities(self, item_vector : np.ndarray):
      """
      item_vector : 1D numpy array
      
      """
      # self.utility_matrix has each column as a user and each row as an item
      utility_rating_means = np.nanmean(self.utility_matrix.T, axis=1)
      item_rating_mean = np.nanmean(item_vector)
      # subtract the mean rating of each user from their ratings
      utility_matrix_centered = self.utility_matrix.T - utility_rating_means[:, np.newaxis]
      
      # compute the cosine similarity between item_vector and each item
      item_vector_centered = item_vector - item_rating_mean
      
      # compute the norms of the vectors, which might have nans as well
      item_vector_norm = np.sqrt(np.nansum(item_vector_centered**2))
      utility_matrix_norm = np.sqrt(np.nansum(utility_matrix_centered**2, axis=1))
      
      # compute the dot product between item_vector and each item
      dot_product = np.nansum(utility_matrix_centered *item_vector_centered, axis=1)
      
      # compute the cosine similarity
      similarities = dot_product/(item_vector_norm * utility_matrix_norm + self.epsilon)
      
      return similarities
    
    
    def find_k_nearest_items(self, item_vector : np.ndarray, u: int):
      """
      item_vector : 1D numpy array
      
      returns the indices of the k most similar items to item_vector
      """
      
      similarities = self.compute_similarities(item_vector)
      items_to_consider = np.where(~np.isnan(self.utility_matrix[u]))[0]
      k = min(self.k, len(items_to_consider))
      sorted_indices = np.argsort(similarities[items_to_consider])[-k:]
      items_to_consider = items_to_consider[sorted_indices]
      similarities = similarities[items_to_consider]
      return similarities, items_to_consider
    
    
    def predict(self, item_vector : np.ndarray, u: int):
      """
      item_vector : 1D numpy array
      u : int
      u is the index of the user to predict the rating for
      """
      similarity_scores, items_to_consider = self.find_k_nearest_items(item_vector, u)
      
      # predict the rating for the item
      prediction = self.final_rating_function(self.utility_matrix[u, items_to_consider], similarity_scores)
      return prediction

In [50]:
def test_item_item_filter(train_matrix: np.ndarray, expert_matrix: np.ndarray, user_start: int, item_start: int, model: ItemBasedCollaborativeFilter) -> float:
    rmse_loss = 0
    count = 0
    for i in range(user_start, expert_matrix.shape[0]):
        for j in range(item_start, expert_matrix.shape[1]):
            if not np.isnan(expert_matrix[i, j]):
                prediction = model.predict(train_matrix[:, j], i)
                # print(prediction, i, j)
                rmse_loss += (prediction - expert_matrix[i, j])**2
                count += 1
    rmse_loss = np.sqrt(rmse_loss/count)
    
    return rmse_loss

In [51]:
item_cf = ItemBasedCollaborativeFilter(train_matrix, 5, 'weighted')
# test_item_item_filter(train_matrix, expert_matrix, test_users_start, test_tags_start, item_cf)

In [52]:
item_cf = ItemBasedCollaborativeFilter(train_matrix, 5, 'regular_average')
# test_item_item_filter(train_matrix, expert_matrix, test_users_start, test_tags_start, item_cf)

# Part 5

In [53]:
class LatentFactorDecomposition:
    def __init__(self, utility_matrix: np.ndarray, f : int, regp :int= 0, regq:int = 0):
        """
        utility_matrix : 2D numpy array, Expert matrix
        f : Number of Latent Factors
        regp: regularization rate over p
        regq: regularization rate over q
        """
        self.utility_matrix = utility_matrix
        self.f = f
        self.epsilon = 1e-9
        self.regp = regp
        self.regq = regq
        num_users, num_items = utility_matrix.shape
        self.P = np.random.randn(num_users, f)/np.sqrt(num_users*f)
        self.Q = np.random.randn(num_items, f)/np.sqrt(num_items*f)
    
    def predict(self, u, i):
        return np.dot(self.P[u], self.Q[i])
    
    def train(self, epochs = 1000, thresh = 0.005, learn_start = 0.001):
        
        prev_loss = np.inf
        alpha = learn_start
        for epoch in range(epochs):
            pred_matrix = np.dot(self.P, self.Q.T)
            diffs = pred_matrix - self.utility_matrix
            loss = np.nansum(diffs**2) + self.regp*np.nansum(self.P**2) + self.regq*np.nansum(self.Q**2)
            
            # applying SGD
            p_reg_changes = 2*self.regp*self.P 
            q_reg_changes = 2*self.regq*self.Q
            
            
            diffs[np.isnan(diffs)] = 0
            # from observation each p[i,j] affects prediction of only user i for each item k. So, its SGD will get changes for user i and each item k.
            p_rmse_changes = 2*(np.dot(diffs, self.Q))
            # from observation each q[i,j] affects prediction of only item i for each user k. So, its SGD will get changes for item i and each user k.
            q_rmse_changes = 2*(np.dot(diffs.T, self.P))
            
            self.P -= alpha*(p_rmse_changes + p_reg_changes)
            self.Q -= alpha*(q_rmse_changes + q_reg_changes)
            if(prev_loss - loss < thresh and prev_loss - loss > 0):
                alpha /= 10
            if epoch% 50 == 0:
                print(f"Epoch {epoch+1}/{epochs} : Loss = {loss}")
            
    def get_factors(self):
        return self.P, self.Q

In [54]:
def test_lfd(model: LatentFactorDecomposition, expert_matrix: np.ndarray, user_start: int, item_start : int,f = 5):
    rmse_loss = 0
    count = 0
    for i in range(user_start, expert_matrix.shape[0]):
        for j in range(item_start, expert_matrix.shape[1]):
            if not np.isnan(expert_matrix[i, j]):
                prediction = model.predict(i, j)
                # print(prediction, i, j)
                rmse_loss += (prediction - expert_matrix[i, j])**2
                count += 1
    rmse_loss = np.sqrt(rmse_loss/count)
    return rmse_loss

In [55]:
model = LatentFactorDecomposition(train_matrix, f = 1)
# model.train(epochs=2000)

So, learning is occuring

In [56]:
# test_lfd(model, expert_matrix, test_users_start, test_tags_start)

## Need to complete all deliverables for 4th part and 5th part

## Part-6

In [ ]:
!pip install scikit-learn scikit-surprise

In [87]:
from surprise.model_selection import train_test_split
from surprise import Dataset, Reader
dataset_df = pd.DataFrame(train_matrix)
long_df = dataset_df.stack().reset_index()
long_df.columns = ['user_id', 'item_id', 'rating']
# Define reader with rating scale (adjust if necessary)
reader = Reader(rating_scale=(0, 5))

# Load into Surprise dataset
train_data = Dataset.load_from_df(long_df, reader)
# Build the full training set
trainset = train_data.build_full_trainset()




In [ ]:
test_users_start, test_tags_start

In [ ]:
test_matrix = np.copy(expert_matrix)
test_df = pd.DataFrame(test_matrix)
test_long_df = test_df.stack().reset_index()
test_long_df.columns = ['user_id', 'item_id', 'rating']
test_df_final = test_long_df[(test_long_df['user_id'] >= test_users_start) & (test_long_df['item_id'] >= test_tags_start)].reset_index(drop=True)
test_data = Dataset.load_from_df(test_df_final, reader)
testset = [tuple(x) for x in test_df_final[['user_id', 'item_id', 'rating']].values]
testset[:10]

In [ ]:
from surprise.prediction_algorithms.knns import KNNBasic

model = KNNBasic(sim_options = {'name': 'pearson', 'user_based': False}, k =5)
model.fit(trainset)

In [ ]:
from surprise import accuracy
predictions = model.test(testset)
rmse = accuracy.rmse(predictions)

In [ ]:
from surprise import Dataset, SVD

algo = SVD()

algo.fit(trainset)


In [ ]:
predictions = algo.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)